# Point History Classifier Training
Matches the dynamic gesture workflow with optional LSTM support.

In [ ]:
import csv
from pathlib import Path

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
RANDOM_SEED = 42
NUM_CLASSES = 4
TIME_STEPS = 16
DIMENSION = 2
DATASET_PATH = Path('../data/point_history.csv')
MODEL_SAVE_PATH = Path('../models/point_history/point_history_classifier.hdf5')
TFLITE_SAVE_PATH = Path('../models/point_history/point_history_classifier.tflite')
use_lstm = False

In [ ]:
X_dataset = np.loadtxt(DATASET_PATH, delimiter=',', dtype='float32', usecols=list(range(1, (TIME_STEPS * DIMENSION) + 1)))
y_dataset = np.loadtxt(DATASET_PATH, delimiter=',', dtype='int32', usecols=(0,))

X_train, X_test, y_train, y_test = train_test_split(
    X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED
)

In [ ]:
if use_lstm:
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(TIME_STEPS * DIMENSION,)),
        tf.keras.layers.Reshape((TIME_STEPS, DIMENSION)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(16),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'),
    ])
else:
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(TIME_STEPS * DIMENSION,)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'),
    ])

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(str(MODEL_SAVE_PATH), save_best_only=True)
early_stop_cb = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[checkpoint_cb, early_stop_cb],
)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()
TFLITE_SAVE_PATH.write_bytes(tflite_quantized_model)